In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import os
import sys

Append the `grt_reader` repo so we can read modules from sibling directories.


In [13]:
sys.path.append(os.path.abspath(os.path.join("..")))

In [14]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from utils.constants import question_ids_map
from utils.experiment import Experiment
from utils.process_lease import process_lease
from utils.questions import (
    construct_single_question_for_ai,
)

## Threshold for Levenshtein distance


In [15]:
threshold = 80

Retrieve questions


In [16]:
questions_df = pd.read_excel("../data/asc_842/grt_ai_reader_questions_v1.xlsx")
questions_df = questions_df.drop(["encoding"], axis=1)
questions_df = questions_df.replace({np.nan: None})

Retrieve answers file


In [17]:
answers_df = pd.read_excel("../data/asc_842/lease_agreements_info/answers.xlsx")

Get questions to send to the AI


In [18]:
question_id = question_ids_map["lessor_name"]
question_df_filtered = questions_df[questions_df["id"] == question_id]
question_as_json = question_df_filtered.to_dict(orient="records")[0]
question_for_ai = construct_single_question_for_ai(question_as_json)

## Experiment 1:

- gpt-3.5-turbo
- text-embedding-ada-002
- unstructured outputs


In [19]:
experiment = Experiment(
    answers_df=answers_df,
    question_id=question_id,
    question_for_ai=question_for_ai,
    threshold=threshold,
    model="gpt-3.5-turbo",
    embedding_model="text-embedding-ada-002",
    already_stored=True,
    already_vectorized=True,
    use_structured_outputs=False,
    index_name=os.getenv("INDEX_NAME", ""),
    lessor_question=True,
    csv_results_filename="results/lessor_name_experiment1.csv",
)
experiment.run()

Answer: BarBell Real Estate
Real answer: BarBell Real Estate, LLC
CORRECT
----------------------------------------------------
Answer: Rashed Mujib Noman
Real answer: Sony Chocolate Industries Ltd
INCORRECT
----------------------------------------------------
Answer: Licensor
Real answer: The Wheelership LLC
INCORRECT
----------------------------------------------------
Answer: Landlord
Real answer: Venable Tenant, LLC
INCORRECT
----------------------------------------------------
Answer: Landlord
Real answer: Wyomissing Professional Center III Limited Partnership
INCORRECT
----------------------------------------------------
Answer: Lessor
Real answer: Hooten Non Exempt Family Trust B
INCORRECT
----------------------------------------------------
Answer: Landlord
Real answer: AR Industrial No. 1, Ltd
INCORRECT
----------------------------------------------------
Answer: Pennsylvania Department of Banking
Real answer: Village Corner, LLC
INCORRECT
--------------------------------------

KeyboardInterrupt: 

In [ ]:
total_answers = 0
correct_answers = 0
for number in numbers_list:
    file_name = f"lease{number}"
    local_file = f"../data/asc_842/lease_agreements/{file_name}.pdf"
    file_key = f"eafit/{file_name}.pdf"
    namespace = f"eafit_{file_name}"

    # Check if the real answer is NaT
    answer_is_not_given = pd.isna(
        answers_df[answers_df["Lease"] == file_name][question_id].iloc[0]
    )

    if answer_is_not_given:
        continue

    total_answers += 1

    try:
        result = process_lease(
            local_file=local_file,
            file_name=file_name,
            file_key=file_key,
            namespace=namespace,
            question_for_ai=question_for_ai,
            question_id=question_id,
            answers_df=answers_df,
        )
        answer = result["answer"]
        real_answer_unprocessed = result["real_answer_unprocessed"]
        formatted_answer = real_answer_unprocessed.iloc[0]
        print(f"Answer: {answer[0]}")
        print(f"Real answer: {formatted_answer}")
        similarity_score = fuzz.ratio(
            answer[0].lower(), real_answer_unprocessed.iloc[0].lower()
        )
        if similarity_score >= threshold:
            correct_answers += 1
            print("CORRECT")
        else:
            print("INCORRECT")
        print("----------------------------------------------------")
    except Exception as error:
        print(f"Error processing {file_name}: {error}")
        continue
accuracy = correct_answers / total_answers
accuracy

## Experiment 7:

- gpt-4o-mini
- text-embedding-3-small
- structured outputs


In [16]:
total_answers = 0
correct_answers = 0

for number in numbers_list:
    file_name = f"lease{number}"
    local_file = f"../data/asc_842/lease_agreements/{file_name}.pdf"
    file_key = f"eafit/{file_name}.pdf"
    namespace = f"eafit_{file_name}"

    # Check if the real answer is NaT
    answer_is_not_given = pd.isna(
        answers_df[answers_df["Lease"] == file_name][question_id].iloc[0]
    )

    if answer_is_not_given:
        continue

    total_answers += 1

    try:
        result = process_lease(
            local_file=local_file,
            file_name=file_name,
            file_key=file_key,
            namespace=namespace,
            question_for_ai=question_for_ai,
            question_id=question_id,
            answers_df=answers_df,
            embedding_model="text-embedding-3-small",
            model="gpt-4o-mini",
        )
        answer = result["answer"]
        real_answer_unprocessed = result["real_answer_unprocessed"]
        formatted_answer = real_answer_unprocessed.iloc[0]
        print(f"Answer: {answer.answer_string[0]}")
        print(f"Real answer: {formatted_answer}")
        similarity_score = fuzz.ratio(
            answer.answer_string[0].lower(), real_answer_unprocessed.iloc[0].lower()
        )
        if similarity_score >= threshold:
            correct_answers += 1
            print("CORRECT")
        else:
            print("INCORRECT")
        print("----------------------------------------------------")
    except Exception as error:
        print(f"Error processing {file_name}: {error}")
        continue

accuracy = correct_answers / total_answers
accuracy

Upload Successful: eafit/lease001.pdf
Stored eafit/lease001.pdf in vector database with namespace eafit_lease001
Answer: BarBell Real Estate & HVE, Inc.
Real answer: BarBell Real Estate, LLC
CORRECT
----------------------------------------------------
Upload Successful: eafit/lease002.pdf
Stored eafit/lease002.pdf in vector database with namespace eafit_lease002
Answer: The name of the lessor in the agreement is the LESSOR.
Real answer: Sony Chocolate Industries Ltd
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease003.pdf
Stored eafit/lease003.pdf in vector database with namespace eafit_lease003
Answer: The Wheelership LLC
Real answer: The Wheelership LLC
CORRECT
----------------------------------------------------
Upload Successful: eafit/lease004.pdf
Stored eafit/lease004.pdf in vector database with namespace eafit_lease004
Answer: Venable Tenant, LLC
Real answer: Venable Tenant, LLC
CORRECT
-------------------------------------------------

0.17

## Experiment 11:

- gpt-4o-2024-08-06
- text-embedding-3-small
- structured outputs


In [13]:
total_answers = 0
correct_answers = 0

for number in numbers_list:
    file_name = f"lease{number}"
    local_file = f"../data/asc_842/lease_agreements/{file_name}.pdf"
    file_key = f"eafit/{file_name}.pdf"
    namespace = f"eafit_{file_name}"

    # Check if the real answer is NaT
    answer_is_not_given = pd.isna(
        answers_df[answers_df["Lease"] == file_name][question_id].iloc[0]
    )

    if answer_is_not_given:
        continue

    total_answers += 1

    try:
        result = process_lease(
            local_file=local_file,
            file_name=file_name,
            file_key=file_key,
            namespace=namespace,
            question_for_ai=question_for_ai,
            question_id=question_id,
            answers_df=answers_df,
            embedding_model="text-embedding-3-small",
            model="gpt-4o-2024-08-06",
        )
        answer = result["answer"]
        real_answer_unprocessed = result["real_answer_unprocessed"]
        formatted_answer = real_answer_unprocessed.iloc[0]
        print(f"Answer: {answer.answer_string[0]}")
        print(f"Real answer: {formatted_answer}")
        similarity_score = fuzz.ratio(
            answer.answer_string[0].lower(), real_answer_unprocessed.iloc[0].lower()
        )
        if similarity_score >= threshold:
            correct_answers += 1
            print("CORRECT")
        else:
            print("INCORRECT")
        print("----------------------------------------------------")
    except Exception as error:
        print(f"Error processing {file_name}: {error}")
        continue

accuracy = correct_answers / total_answers
accuracy

Upload Successful: eafit/lease001.pdf
Stored eafit/lease001.pdf in vector database with namespace eafit_lease001
Answer: BarBell Real Estate
Real answer: BarBell Real Estate, LLC
CORRECT
----------------------------------------------------
Upload Successful: eafit/lease002.pdf
Stored eafit/lease002.pdf in vector database with namespace eafit_lease002
Answer: The lessor is not explicitly named in the provided source document.
Real answer: Sony Chocolate Industries Ltd
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease003.pdf
Stored eafit/lease003.pdf in vector database with namespace eafit_lease003
Answer: The Wheelership LLC
Real answer: The Wheelership LLC
CORRECT
----------------------------------------------------
Upload Successful: eafit/lease004.pdf
Stored eafit/lease004.pdf in vector database with namespace eafit_lease004
Answer: Venable Tenant, LLC
Real answer: Venable Tenant, LLC
CORRECT
------------------------------------------------

0.46

## Experiment 5:

- gpt-3.5-turbo
- text-embedding-3-small
- structured outputs


In [27]:
total_answers = 0
correct_answers = 0

for number in numbers_list:
    file_name = f"lease{number}"
    local_file = f"../data/asc_842/lease_agreements/{file_name}.pdf"
    file_key = f"eafit/{file_name}.pdf"
    namespace = f"eafit_{file_name}"

    # Check if the real answer is NaT
    answer_is_not_given = pd.isna(
        answers_df[answers_df["Lease"] == file_name][question_id].iloc[0]
    )

    if answer_is_not_given:
        continue

    total_answers += 1

    try:
        result = process_lease(
            local_file=local_file,
            file_name=file_name,
            file_key=file_key,
            namespace=namespace,
            question_for_ai=question_for_ai,
            question_id=question_id,
            answers_df=answers_df,
            embedding_model="text-embedding-3-small",
            model="gpt-3.5-turbo",
            force_structured_output=True,
        )
        answer = result["answer"]
        real_answer_unprocessed = result["real_answer_unprocessed"]
        formatted_answer = real_answer_unprocessed.iloc[0]
        print(f"Answer: {answer[0]}")
        print(f"Real answer: {formatted_answer}")
        similarity_score = fuzz.ratio(
            answer[0].lower(), real_answer_unprocessed.iloc[0].lower()
        )
        if similarity_score >= threshold:
            correct_answers += 1
            print("CORRECT")
        else:
            print("INCORRECT")
        print("----------------------------------------------------")
    except Exception as error:
        print(f"Error processing {file_name}: {error}")
        continue

accuracy = correct_answers / total_answers
accuracy

Upload Successful: eafit/lease001.pdf
Stored eafit/lease001.pdf in vector database with namespace eafit_lease001
Answer: Barbell Real Estate & HVE, Inc.
Real answer: BarBell Real Estate, LLC
CORRECT
----------------------------------------------------
Upload Successful: eafit/lease002.pdf
Stored eafit/lease002.pdf in vector database with namespace eafit_lease002
Answer: The lessor in the agreement is referred to as the LESSOR throughout the document.
Real answer: Sony Chocolate Industries Ltd
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease003.pdf
Stored eafit/lease003.pdf in vector database with namespace eafit_lease003
Error processing lease003: '2'
Upload Successful: eafit/lease004.pdf
Stored eafit/lease004.pdf in vector database with namespace eafit_lease004
Answer: Venable Tenant, LLC
Real answer: Venable Tenant, LLC
CORRECT
----------------------------------------------------
Upload Successful: eafit/lease005.pdf
Stored eafit/lease005.

0.35

## Experiment 6:

- gpt-3.5-turbo
- text-embedding-3-large
- structured outputs


In [13]:
total_answers = 0
correct_answers = 0

for number in numbers_list:
    file_name = f"lease{number}"
    local_file = f"../data/asc_842/lease_agreements/{file_name}.pdf"
    file_key = f"eafit/{file_name}.pdf"
    namespace = f"eafit_{file_name}"

    # Check if the real answer is NaT
    answer_is_not_given = pd.isna(
        answers_df[answers_df["Lease"] == file_name][question_id].iloc[0]
    )

    if answer_is_not_given:
        continue

    total_answers += 1

    try:
        result = process_lease(
            local_file=local_file,
            file_name=file_name,
            file_key=file_key,
            namespace=namespace,
            question_for_ai=question_for_ai,
            question_id=question_id,
            answers_df=answers_df,
            embedding_model="text-embedding-3-large",
            model="gpt-3.5-turbo",
            index_name=os.getenv("INDEX_LATEST_NAME", ""),
            force_structured_output=True,
        )
        answer = result["answer"]
        real_answer_unprocessed = result["real_answer_unprocessed"]
        formatted_answer = real_answer_unprocessed.iloc[0]
        print(f"Answer: {answer[0]}")
        print(f"Real answer: {formatted_answer}")
        similarity_score = fuzz.ratio(
            answer[0].lower(), real_answer_unprocessed.iloc[0].lower()
        )
        if similarity_score >= threshold:
            correct_answers += 1
            print("CORRECT")
        else:
            print("INCORRECT")
        print("----------------------------------------------------")
    except Exception as error:
        print(f"Error processing {file_name}: {error}")
        continue

accuracy = correct_answers / total_answers
accuracy

Upload Successful: eafit/lease001.pdf
Stored eafit/lease001.pdf in vector database with namespace eafit_lease001
Answer: Kyle J. Beller
Real answer: BarBell Real Estate, LLC
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease002.pdf
Stored eafit/lease002.pdf in vector database with namespace eafit_lease002
Answer: Augmedix Bangladesh Ltd.
Real answer: Sony Chocolate Industries Ltd
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease003.pdf
Stored eafit/lease003.pdf in vector database with namespace eafit_lease003
Answer: ABC Leasing Corporation
Real answer: The Wheelership LLC
INCORRECT
----------------------------------------------------
Upload Successful: eafit/lease004.pdf
Stored eafit/lease004.pdf in vector database with namespace eafit_lease004
Answer: Venable Tenant, LLC
Real answer: Venable Tenant, LLC
CORRECT
----------------------------------------------------
Upload Successful: eafit/lease005.p

0.2

## Experiment 10:

- gpt-4o-mini
- text-embedding-3-large
- structured outputs


In [12]:
total_answers = 0
correct_answers = 0

for number in numbers_list:
    file_name = f"lease{number}"
    local_file = f"../data/asc_842/lease_agreements/{file_name}.pdf"
    file_key = f"eafit/{file_name}.pdf"
    namespace = f"eafit_{file_name}"

    # Check if the real answer is NaT
    answer_is_not_given = pd.isna(
        answers_df[answers_df["Lease"] == file_name][question_id].iloc[0]
    )

    if answer_is_not_given:
        continue

    total_answers += 1

    try:
        result = process_lease(
            local_file=local_file,
            file_name=file_name,
            file_key=file_key,
            namespace=namespace,
            question_for_ai=question_for_ai,
            question_id=question_id,
            answers_df=answers_df,
            embedding_model="text-embedding-3-large",
            model="gpt-4o-mini",
            index_name=os.getenv("INDEX_LATEST_NAME", ""),
            already_stored=True,
            already_vectorized=True,
        )
        answer = result["answer"]
        real_answer_unprocessed = result["real_answer_unprocessed"]
        formatted_answer = real_answer_unprocessed.iloc[0]
        print(f"Answer: {answer.answer_string[0]}")
        print(f"Real answer: {formatted_answer}")
        similarity_score = fuzz.ratio(
            answer.answer_string[0].lower(), real_answer_unprocessed.iloc[0].lower()
        )
        if similarity_score >= threshold:
            correct_answers += 1
            print("CORRECT")
        else:
            print("INCORRECT")
        print("----------------------------------------------------")
    except Exception as error:
        print(f"Error processing {file_name}: {error}")
        continue

accuracy = correct_answers / total_answers
accuracy

Answer: Kyle J. Beller
Real answer: BarBell Real Estate, LLC
INCORRECT
----------------------------------------------------
Answer: Augmedix Bangladesh Ltd.
Real answer: Sony Chocolate Industries Ltd
INCORRECT
----------------------------------------------------
Answer: The name of the lessor in the agreement is Licensor.
Real answer: The Wheelership LLC
INCORRECT
----------------------------------------------------
Answer: Venable Tenant, LLC
Real answer: Venable Tenant, LLC
CORRECT
----------------------------------------------------
Answer: Marathon Business
Real answer: Wyomissing Professional Center III Limited Partnership
INCORRECT
----------------------------------------------------
Answer: The name of the lessor in the agreement is Lessor.
Real answer: Hooten Non Exempt Family Trust B
INCORRECT
----------------------------------------------------
Answer: The name of the lessor in the agreement is Landlord.
Real answer: AR Industrial No. 1, Ltd
INCORRECT
------------------------

0.41